In [1]:
import requests
import pandas as pd
import math
from datetime import datetime, timedelta
import time
from collections import namedtuple
import os
from arcgis.gis import GIS
from multiprocessing import Process, Queue

from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv())

import sys
sys.path.append('../src')

import reach_tools
import hydrology

%load_ext autoreload
%autoreload 2

In [2]:
reach_id = '3066'

In [3]:
def get_current_observation(reach_id):
    
    Response = namedtuple('Response', ['status', 'observation', 'runnable'])

    aw_resp = requests.get(f'https://www.americanwhitewater.org/content/River/detail/id/{reach_id}/.json')

    if aw_resp.status_code == 200:

        resp_json = aw_resp.json()
        if len(resp_json['CContainerViewJSON_view']['CRiverMainGadgetJSON_main']['gauges']):
            gauge_info = resp_json['CContainerViewJSON_view']['CRiverMainGadgetJSON_main']['gauges'][0]
            gauge_id = gauge_info['gauge_id']
            gauge_metric = gauge_info['gauge_metric']
            gauge_units = gauge_info['metric_unit']
            gauge_min = float(gauge_info['gauge_min'])
            gauge_max = float(gauge_info['gauge_max'])
            gauge_observation = float(gauge_info['gauge_reading'])
            reach_runnable = gauge_min < gauge_observation < gauge_max
            
            gauge_resp = Response(True, gauge_observation, reach_runnable)
            
        else:
            gauge_resp = Respones(False)
    
    else:
        gauge_resp = Response(False)
        
    return gauge_resp

In [4]:
gis = GIS(username=os.getenv('ARCGIS_USERNAME'), password=os.getenv('ARCGIS_PASSWORD'))

url_reach_line = os.getenv('URL_REACH_LINE')
url_reach_centroid = os.getenv('URL_REACH_CENTROID')
url_reach_points = os.getenv('URL_REACH_POINT')

lyr_reach_line = reach_tools.ReachFeatureLayer(url_reach_line, gis)
lyr_reach_centroid = reach_tools.ReachFeatureLayer(url_reach_centroid, gis)
lyr_reach_points = reach_tools.ReachPointFeatureLayer(url_reach_points, gis)

In [5]:
def get_featureset(lyr, reach_id):
    attempts = 0
    where_clause = f"reach_id = '{reach_id}'"
    while attempts < 10:
        try:
            return lyr.query(where_clause, out_fields=['gauge_observation', 'gauge_runnable'], return_geometry=False)
        except:
            attempts = attempts + 1
    

def apply_updates(lyr, update_fs):
    attempts = 0
    while attempts < 10:
        try:
            lyr.edit_features(updates=update_fs)
            break
        except:
            attempts = attempts + 1

def update_reach_stage(reach_id):
    reach_aw = reach_tools.Reach.get_from_aw(reach_id)

    fs_centroid = get_featureset(lyr_reach_centroid, reach_id)
    fs_line = get_featureset(lyr_reach_line, reach_id)
    
    if reach_aw.gauge_observation:

        for fs in [fs_centroid, fs_line]:
            if len(fs.features):
                for feature in fs.features:
                    feature.attributes['gauge_observation'] = reach_aw.gauge_observation
                    feature.attributes['gauge_runnable'] = reach_aw.gauge_runnable

        apply_updates(lyr_reach_centroid, fs_centroid)
        if len(fs_line.features):
            apply_updates(lyr_reach_line, fs_line)
            
        print(f'updated: {reach_id}')
    else:
        print(f'no stage info for: {reach_id}')
        
    last_reach = reach_id
    
def process_data(q):
    reach_id = q.get()
    update_reach_stage(reach_id)

In [6]:
fs_centroid_unique = lyr_reach_centroid.query(out_fields='reach_id', return_distinct_values=True, return_geometry=False)
centroid_reach_id_list = [f.attributes['reach_id'] for f in fs_centroid_unique.features]
centroid_reach_id_list = [int(reach_id) for reach_id in centroid_reach_id_list]
centroid_reach_id_list.sort()
last_reach = 0
centroid_reach_id_list[:10]

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

In [10]:
centroid_reach_id_list = [val for val in centroid_reach_id_list if val > 2231]
print(len(centroid_reach_id_list))

3513


In [11]:
q = Queue()
processes = [Process(target=process_data, args=(q,)) for i in range(8)]

for p in processes:
    p.start()

for reach_id in centroid_reach_id_list:
    q.put(reach_id)
    
for p in processes:
    p.join()

no stage info for: 2236
no stage info for: 2235
updated: 2239
no stage info for: 2238
no stage info for: 2232
updated: 2233
updated: 2234
updated: 2237


In [ ]:
for reach_id in centroid_reach_id_list:
    update_reach_stage(reach_id)

no stage info for: 2232
updated: 2233
updated: 2234
no stage info for: 2235
no stage info for: 2236
updated: 2237
no stage info for: 2238
updated: 2239
updated: 2240
updated: 2241
updated: 2242
updated: 2243
updated: 2244
updated: 2245
updated: 2246
updated: 2247
no stage info for: 2248
updated: 2249
updated: 2250
updated: 2251
updated: 2252
updated: 2253
updated: 2254
no stage info for: 2255
updated: 2256
no stage info for: 2257
updated: 2258
no stage info for: 2259
updated: 2260
updated: 2261
updated: 2262
updated: 2263
updated: 2264
updated: 2265
updated: 2266
updated: 2267
updated: 2268
updated: 2269
no stage info for: 2270
no stage info for: 2271
no stage info for: 2272
updated: 2273
updated: 2274
updated: 2275
updated: 2276
updated: 2277
updated: 2278
no stage info for: 2279
updated: 2280
updated: 2281
updated: 2282
updated: 2283
no stage info for: 2284
no stage info for: 2285
updated: 2286
updated: 2287
updated: 2288
updated: 2289
no stage info for: 2290
no stage info for: 2291


An error occurred.


updated: 3740
updated: 3741
updated: 3742
updated: 3743
no stage info for: 3744
updated: 3745
updated: 3746
updated: 3747
no stage info for: 3748
updated: 3749
updated: 3750
updated: 3751
updated: 3752
updated: 3753
updated: 3754
updated: 3755
updated: 3756
no stage info for: 3757
updated: 3758
updated: 3759
no stage info for: 3760
updated: 3761
no stage info for: 3762
updated: 3763
updated: 3764
updated: 3765
no stage info for: 3766
updated: 3767
no stage info for: 3768
no stage info for: 3769
no stage info for: 3770
updated: 3771
no stage info for: 3772
updated: 3773
updated: 3774
updated: 3775
updated: 3776
updated: 3780
updated: 3781
updated: 3782
updated: 3783
updated: 3784
no stage info for: 3785
updated: 3786
updated: 3787
updated: 3788
updated: 3789
updated: 3790
no stage info for: 3791
updated: 3792
no stage info for: 3793
updated: 3794
no stage info for: 3795
updated: 3796
updated: 3797
updated: 3798
updated: 3799
updated: 3800
updated: 3801
updated: 3802
no stage info for: 3

An error occurred.


updated: 3887
no stage info for: 3888
no stage info for: 3889
updated: 3890
updated: 3891
updated: 3892
no stage info for: 3893
no stage info for: 3894
updated: 3895
updated: 3896
no stage info for: 3897
updated: 3898
no stage info for: 3899
no stage info for: 3900
updated: 3901
no stage info for: 3902
no stage info for: 3903
updated: 3904
updated: 3905
updated: 3906
no stage info for: 3907
updated: 3909
updated: 3910
updated: 3911
no stage info for: 3912
no stage info for: 3913
no stage info for: 3914
no stage info for: 3915
no stage info for: 3916
updated: 3917
updated: 3918
updated: 3919
updated: 3920
updated: 3921
updated: 3922
updated: 3923
updated: 3924
updated: 3925
updated: 3926
no stage info for: 3927
updated: 3928
updated: 3929
no stage info for: 3930
no stage info for: 3931
no stage info for: 3932
no stage info for: 3933
no stage info for: 3934
no stage info for: 3935
updated: 3936
updated: 3937
no stage info for: 3938
no stage info for: 3939
no stage info for: 3940
no stage

In [51]:
last_reach

0

In [5]:
endTime = datetime.now()
startTime = endTime - timedelta(days=1)

endTimeStamp = int(time.mktime(endTime.timetuple()))
startTimeStamp = int(time.mktime(startTime.timetuple()))

url = f'https://www.americanwhitewater.org/api/gauge/{gauge_id}/flows/{gauge_metric}'

params = {
    'from': startTimeStamp,
    'to': endTimeStamp,
    'resolution': 3600
}

resp = requests.get(url, params=params)

df = pd.DataFrame.from_dict(resp.json())

df.updated = df.updated.apply(lambda val: datetime.fromtimestamp(round(float(val))))
df.head()

NameError: name 'gauge_id' is not defined